In [125]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from scipy import stats
import math
import statistics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize


# Split datasets into train and test datasets
%matplotlib inline

df_final = pd.read_csv('dat_final.csv')
y = df_final['price_sqm']
X = df_final[['floor_area_sqm', 'remaining_lease', 'year','distance_to_downtown', 'distance_to_nearest_mrt',
        'town_ANG MO KIO','toilet_number',
       'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK', 'town_BUKIT MERAH',
       'town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA',
       'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG',
       'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA',
       'town_LIM CHU KANG', 'town_MARINE PARADE', 'town_PASIR RIS',
       'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
       'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
       'town_YISHUN', 'direction_CENTRAL', 'direction_EAST', 'direction_NORTH',
       'direction_NORTH_EAST', 'direction_WEST', 
       'flat_model_2-room', 'flat_model_Adjoined flat', 'flat_model_Apartment',
       'flat_model_DBSS', 'flat_model_Improved', 'flat_model_Maisonette',
       'flat_model_Model A', 'flat_model_Model A2',
       'flat_model_Multi Generation', 'flat_model_New Generation',
       'flat_model_Premium Apartment', 'flat_model_STANDARD',
       'flat_model_Simplified', 'flat_model_Standard', 'flat_model_Terrace',
       'flat_model_Type S1', 'flat_model_Type S2', 'storey_avg']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.25, random_state = 2020)

In [133]:
df_final

,Unnamed: 0,floor_area_sqm,remaining_lease,year,price_sqm,Unnamed: 0.1,distance_to_downtown,distance_to_nearest_mrt,Bus,Food Court,...,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2,storey_avg,storey < 10,storey < 15,storey < 20,storey < 30,story > 30
0,0,44.0,61,2017,5272.727273,0,8.383132,0.851339,0.0,5.0,...,0,0,0,0,12,0,1,0,0,0
1,1,67.0,60,2017,3731.343284,1,9.236619,0.696968,0.0,2.0,...,0,0,0,0,3,1,0,0,0,0
2,2,67.0,62,2017,3910.447761,2,8.911143,0.617914,2.0,1.0,...,0,0,0,0,3,1,0,0,0,0
3,3,68.0,62,2017,3897.058824,0,8.383132,0.851339,0.0,5.0,...,0,0,0,0,6,1,0,0,0,0
4,4,67.0,62,2017,3955.223881,2,8.911143,0.617914,2.0,1.0,...,0,0,0,0,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839605,839605,142.0,69,1999,3211.267606,357,13.132845,0.637828,1.0,2.0,...,0,0,0,0,12,0,1,0,0,0
839606,839606,142.0,70,1999,2873.239437,360,14.026676,0.411571,1.0,3.0,...,0,0,0,0,3,1,0,0,0,0
839607,839607,146.0,70,1999,3212.328767,358,14.230671,1.670481,0.0,0.0,...,0,0,0,0,9,1,0,0,0,0
839608,839608,146.0,70,1999,3013.698630,348,13.932028,1.254211,0.0,3.0,...,0,0,0,0,6,1,0,0,0,0


In [127]:
scaler = MinMaxScaler().fit(X_train)
X_trains = scaler.transform(X_train)
X_tests = scaler.transform(X_test)

#### Scale the data

In [128]:
X_trains

array([[2.72401434e-01, 3.39622642e-01, 7.67136350e-04, ...,
        0.00000000e+00, 0.00000000e+00, 6.25000000e-02],
       [3.76344086e-01, 3.39622642e-01, 3.40552054e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.14695341e-01, 3.77358491e-01, 7.54446763e-04, ...,
        0.00000000e+00, 0.00000000e+00, 6.25000000e-02],
       ...,
       [1.07526882e-02, 1.69811321e-01, 9.12218079e-05, ...,
        0.00000000e+00, 0.00000000e+00, 1.87500000e-01],
       [3.40501792e-01, 3.58490566e-01, 9.26974826e-04, ...,
        0.00000000e+00, 0.00000000e+00, 1.87500000e-01],
       [4.33691756e-01, 5.09433962e-01, 1.07119229e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [117]:
X_trains

array([[ 0.31931241, -0.34422483, -0.67241312, ..., -0.01755491,
        -0.01309724, -0.55344605],
       [ 1.43379936, -0.34422483, -1.04447486, ..., -0.01755491,
        -0.01309724, -1.20011617],
       [-1.3716333 , -0.13511849, -0.30035138, ..., -0.01755491,
        -0.01309724, -0.55344605],
       ...,
       [-2.48612025, -1.28520338,  0.4437721 , ..., -0.01755491,
        -0.01309724,  0.73989419],
       [ 1.04949351, -0.23967166, -0.92045428, ..., -0.01755491,
        -0.01309724,  0.73989419],
       [ 2.04868871,  0.59675372, -0.67241312, ..., -0.01755491,
        -0.01309724, -1.20011617]])

#### Lasso

In [97]:
# lasso = Lasso()
# param = {'alpha':[0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 1, 1.5, 2]}

# lasso_cv = GridSearchCV(lasso,
#                        param,
#                        scoring = 'neg_mean_squared_error',
#                        cv = 5,
#                        verbose = 0)

In [98]:
# lasso_cv.fit(X_trains,y_train)

In [99]:
# best_alpha = pd.Series(lasso_cv.best_params_)[0]

# print(lasso_cv.best_score_)
# print(lasso_cv.best_params_)

In [129]:
lasso = Lasso(alpha = 0.001, 
              random_state = 3000)
lasso = lasso.fit(X_trains, y_train)

/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 116181823446.15741, tolerance: 104434646.31011231
  model = cd_fast.enet_coordinate_descent(


In [130]:
yhat = lasso.predict(X_trains)
MAE_lasso = round(mean_absolute_error(y_train, yhat),2)
RMSE_lasso = round(math.sqrt(mean_squared_error(y_train, yhat)),2)
R2_lasso = round(r2_score(y_train, yhat),2)


print('-------------- Lasso Regression Model on training set------------')
print('MAE: ' + str(mean_absolute_error(y_train, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_train, yhat))))
print('R^2 Score: ' + str(r2_score(y_train, yhat)))

-------------- Lasso Regression Model on training set------------
MAE: 825.5394761751876
RMSE: 1063.5789038049516
R^2 Score: 0.317923086970643


In [83]:
yhat = lasso.predict(X_tests)
MAE_lasso = round(mean_absolute_error(y_test, yhat),2)
RMSE_lasso = round(math.sqrt(mean_squared_error(y_test, yhat)),2)
R2_lasso = round(r2_score(y_test, yhat),2)


print('-------------- Lasso Regression Model ------------')
print('MAE: ' + str(mean_absolute_error(y_test, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_test, yhat))))
print('R^2 Score: ' + str(r2_score(y_test, yhat)))




-------------- Lasso Regression Model ------------
MAE: 546.5991631153242
RMSE: 690.6293625364071
R^2 Score: 0.7143088609329682


In [40]:
y_test - yhat

345091    -552.906868
651411     775.289719
186589      34.961115
811193    -105.358735
705749    1009.866208
             ...     
86861     -363.795347
13914     -815.243169
19365     -305.863478
362418    -400.426006
436629    -537.062926
Name: price_sqm, Length: 209903, dtype: float64

In [37]:
yhat

array([3029.09734461, 1806.67749412, 2712.29163242, ..., 5000.98542946,
       2723.9554181 , 3570.39625924])

In [34]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold

# Convert datasets to XGBDMatrix
dtrain = xgb.DMatrix(X_trains, label = y_train)
dtest = xgb.DMatrix(X_tests)


In [35]:
# Define model parameters
xgb_params = {'obj':'reg:squarederror',
              'eval_metric':'rmse',
              'max_depth':6, 
              'eta':0.5,
              'min_child_weight':1,
              'subsample':1,
              'colsample_bytree':1}

# Find model parameters 'num_boost_round' using cv
cv_res = xgb.cv(xgb_params, 
                dtrain, 
                num_boost_round=500, 
                nfold=5, 
                seed=2020, 
                stratified=False,
                early_stopping_rounds=25, 
                verbose_eval=10, 
                show_stdv=True)

# Get best num_boost_round
nrounds = cv_res[cv_res['test-rmse-mean'] == cv_res['test-rmse-mean'].min()].index[0]
cv_res[cv_res['test-rmse-mean'] == cv_res['test-rmse-mean'].min()]

[03:44:52] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:44:56] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[03:45:00] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are 

KeyboardInterrupt: 

In [ ]:
xgb_def = xgb.train(params = xgb_params,
                   num_boost_round = nrounds,
                   dtrain = dtrain)

In [ ]:
yhat = xgb_def.predict(dtest)
print('-------------- XGB Model: Baseline Parameters ------------')
print('MAE: ' + str(mean_absolute_error(y_test, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_test, yhat))))
print('R^2 Score: ' + str(r2_score(y_test, yhat)))